# Amazon Neptune图存储


In [ ]:
%pip install boto3
%pip install llama-index-llms-bedrock
%pip install llama-index-graph-stores-neptune
%pip install llama-index-embeddings-bedrock

## 使用 NeptuneDatabaseGraphStore 与知识图谱


### 添加所需的导入


In [ ]:
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core import (
    StorageContext,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    Settings,
)
from llama_index.graph_stores.neptune import (
    NeptuneAnalyticsGraphStore,
    NeptuneDatabaseGraphStore,
)
from IPython.display import Markdown, display

### 配置LLM使用的内容，这里是亚马逊Bedrock和Claude 2.1


In [ ]:
llm = Bedrock(model="anthropic.claude-v2")
embed_model = BedrockEmbedding(model="amazon.titan-embed-text-v1")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

### 构建知识图谱

### 读取示例文件


In [ ]:
documents = SimpleDirectoryReader(
    "../../../../examples/paul_graham_essay/data"
).load_data()

### 实例化 Neptune KG 索引

在使用 Amazon Neptune 时，您可以选择使用 Neptune 数据库或 Neptune Analytics。

Neptune 数据库是一种无服务器图数据库，旨在实现最佳的可伸缩性和可用性。它为需要扩展到每秒 100,000 个查询、多可用区高可用性和多区域部署的图数据库工作负载提供了解决方案。您可以将 Neptune 数据库用于社交网络、欺诈警报和客户 360 应用程序。

Neptune Analytics 是一种分析数据库引擎，可以快速分析大量的图数据以获取见解并发现趋势。Neptune Analytics 是快速分析现有图数据库或存储在数据湖中的图数据集的解决方案。它使用流行的图分析算法和低延迟的分析查询。

#### 使用 Neptune 数据库
如果您选择使用 [Neptune 数据库](https://docs.aws.amazon.com/neptune/latest/userguide/feature-overview.html) 来存储您的 KG 索引，您可以按照下面的示例创建图存储。


In [ ]:
graph_store = NeptuneDatabaseGraphStore(
    host="<GRAPH NAME>.<CLUSTER ID>.<REGION>.neptune.amazonaws.com", port=8182
)

#### Neptune Analytics

如果您选择使用[Neptune Analytics](https://docs.aws.amazon.com/neptune-analytics/latest/userguide/what-is-neptune-analytics.html)来存储您的知识图谱索引，您可以按照下面所示的方式创建图形存储。


In [ ]:
graph_store = NeptuneAnalyticsGraphStore(
    graph_identifier="<INSERT GRAPH IDENIFIER>"
)

In [ ]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# 注意：可能需要一些时间！
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

#### 查询知识图谱

首先，我们可以查询并将三元组发送到LLM。


In [ ]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)

response = query_engine.query("Tell me more about Interleaf")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

对于更详细的答案，我们还可以发送从中提取出检索到的三元组的文本。


In [ ]:
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)
response = query_engine.query(
    "Tell me more about what the author worked on at Interleaf"
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))